# This file illustrates how to use the trained models to predict the property of a new molecule

In [83]:
#Set the current directory to the downloaded file
file_path=r"C:\Users\cheese_cake\Desktop\jenny\程序\22_12product design"

#set the property(see variable "name" below,abbreviation is the same as that in paper)
index=0

#set the group-contribution representation of the molecule
mole=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
mole=np.array(mole)

In [84]:
import os  
import warnings
os.chdir(file_path+"\\3prediction")  
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
import numpy as np
from ProductDesignFunction import RMSE,R_square
os.chdir(file_path)
name=["tb","vc","tc","pc","ait",
  "Bcf","Gf","Hf","Hfus","Hsolp",
  "Hv","Lc50_fm","Ld50","Lmv","Logp",
  "Logws","Osha_twa","Pco","Pka","Tm"]
#load the model
model_path="2model/allmodel_"+str(index)+"_"+name[index]+".sav" 
model = pickle.load(open(model_path, 'rb'))

### 1.SVR model
model["simple_model_coef"] and model["simple_model_intercept"] is the coefficient and the intercept

In [85]:
predict_SVR=mole@model["simple_model_coef"]+model["simple_model_intercept"]
print("prediction value of SVR model for property "+name[index]+" is:",predict_SVR)

prediction value of SVR model for property tb is: 556.5657096906646


### 2.GP model
model["normal_model"] is the normal Gaussian Process model

predict both values and uncertainties

In [86]:
test_pre,test_std=model["normal_model"].predict(mole.reshape(1,-1),1)
print("prediction value of GP model for property "+name[index]+" is:",test_pre)
print("uncertainty of GP model for property "+name[index]+" is:",test_std)

prediction value of GP model for property tb is: [516.59389161]
uncertainty of GP model for property tb is: [0.89822655]


### 3.GP-WP model
model["distort_model"] is the model only with a warping function, with model["distort_alpha"] being the hyperparameter in the warping function

model["prior_model"] is the model only with the non-zero prior

model["combination_model"] is the model with warping function and non-zero prior ,with model["combination_alpha"] being the hyperparameter in the warping function

predict both values and uncertainties

In [87]:
#for warping function only
test_input_distort=np.log(mole.reshape(1,-1)+1)/np.log(model["distort_alpha"]) 
test_pre,test_std=model["distort_model"].predict(test_input_distort,1)
print("prediction value of GP-WP model1 for property "+name[index]+" is:",test_pre)
print("uncertainty of GP-WP model1 for property "+name[index]+" is:",test_std)

#for non-zero prior only
test_pre=model["prior_model"].predict(mole.reshape(1,-1))+mole@model["simple_model_coef"]+model["simple_model_intercept"]
test_std=model["prior_model"].predict(mole.reshape(1,-1),1)[1]
print("prediction value of GP-WP model2 for property "+name[index]+" is:",test_pre)
print("uncertainty of GP-WP model2 for property "+name[index]+" is:",test_std)


#for conbination(with non-zero prior and warping function)
test_input_distort=np.log(mole.reshape(1,-1)+1)/np.log(model["combination_alpha"])
test_pre=model["combination_model"].predict(test_input_distort)+mole@model["simple_model_coef"]+model["simple_model_intercept"]
test_std=model["combination_model"].predict(test_input_distort,1)[1]
print("prediction value of GP-WP model3 for property "+name[index]+" is:",test_pre)
print("uncertainty of GP-WP model3 for property "+name[index]+" is:",test_std)

prediction value of GP-WP model1 for property tb is: [530.90093007]
uncertainty of GP-WP model1 for property tb is: [1.0421155]
prediction value of GP-WP model2 for property tb is: [555.26740013]
uncertainty of GP-WP model2 for property tb is: [1.3222979]
prediction value of GP-WP model3 for property tb is: [550.72418265]
uncertainty of GP-WP model3 for property tb is: [1.14992407]
